### Here I am going to make use of Spark SQL where we will not need the normal spark (scala) dataframe functions to transfrom our dataset and provide an output but we will write those Queries in SQL

In [1]:
from pyspark.sql import SparkSession

In [2]:
data_uri = "gs://pysparkbucket-jm/bikeshare.csv"

In [3]:
spark = SparkSession.builder.appName("Read_BikeShare_Data").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/29 15:08:47 INFO SparkEnv: Registering MapOutputTracker
24/03/29 15:08:47 INFO SparkEnv: Registering BlockManagerMaster
24/03/29 15:08:47 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
24/03/29 15:08:47 INFO SparkEnv: Registering OutputCommitCoordinator


In [4]:
df_bikeshare = spark.read.csv(data_uri, header=True, inferSchema=True)

In [5]:
df_bikeshare.printSchema()

root
 |-- datetime: timestamp (nullable = true)
 |-- season: integer (nullable = true)
 |-- holiday: integer (nullable = true)
 |-- workingday: integer (nullable = true)
 |-- weather: integer (nullable = true)
 |-- temp: double (nullable = true)
 |-- atemp: double (nullable = true)
 |-- humidity: integer (nullable = true)
 |-- windspeed: double (nullable = true)
 |-- casual: integer (nullable = true)
 |-- registered: integer (nullable = true)
 |-- count: integer (nullable = true)



In [6]:
df_bikeshare.show(5)

+-------------------+------+-------+----------+-------+----+------+--------+---------+------+----------+-----+
|           datetime|season|holiday|workingday|weather|temp| atemp|humidity|windspeed|casual|registered|count|
+-------------------+------+-------+----------+-------+----+------+--------+---------+------+----------+-----+
|2011-01-01 00:00:00|     1|      0|         0|      1|9.84|14.395|      81|      0.0|     3|        13|   16|
|2011-01-01 01:00:00|     1|      0|         0|      1|9.02|13.635|      80|      0.0|     8|        32|   40|
|2011-01-01 02:00:00|     1|      0|         0|      1|9.02|13.635|      80|      0.0|     5|        27|   32|
|2011-01-01 03:00:00|     1|      0|         0|      1|9.84|14.395|      75|      0.0|     3|        10|   13|
|2011-01-01 04:00:00|     1|      0|         0|      1|9.84|14.395|      75|      0.0|     0|         1|    1|
+-------------------+------+-------+----------+-------+----+------+--------+---------+------+----------+-----+
o

In [7]:
# Create a temporary view of the DataFrame
df_bikeshare.createOrReplaceTempView("bikeshare_data")

In [8]:
sql_query = """
SELECT * 
FROM bikeshare_data
LIMIT 10
"""

In [9]:
result = spark.sql(sql_query)

Same output as df_bikeshare.show()

In [10]:
result.show()

+-------------------+------+-------+----------+-------+-----+------+--------+---------+------+----------+-----+
|           datetime|season|holiday|workingday|weather| temp| atemp|humidity|windspeed|casual|registered|count|
+-------------------+------+-------+----------+-------+-----+------+--------+---------+------+----------+-----+
|2011-01-01 00:00:00|     1|      0|         0|      1| 9.84|14.395|      81|      0.0|     3|        13|   16|
|2011-01-01 01:00:00|     1|      0|         0|      1| 9.02|13.635|      80|      0.0|     8|        32|   40|
|2011-01-01 02:00:00|     1|      0|         0|      1| 9.02|13.635|      80|      0.0|     5|        27|   32|
|2011-01-01 03:00:00|     1|      0|         0|      1| 9.84|14.395|      75|      0.0|     3|        10|   13|
|2011-01-01 04:00:00|     1|      0|         0|      1| 9.84|14.395|      75|      0.0|     0|         1|    1|
|2011-01-01 05:00:00|     1|      0|         0|      2| 9.84| 12.88|      75|   6.0032|     0|         1

Perfroming an operation in pyspark using SQL

In [11]:
sql_query = """
SELECT season, AVG(temp) AS avg_temperature
FROM bikeshare_data
GROUP BY season
"""

It calculates the average temperature for each season in the bikeshare_data DataFrame using Spark SQL

In [12]:
result = spark.sql(sql_query)
result.show()

+------+------------------+
|season|   avg_temperature|
+------+------------------+
|     1|12.530491437081146|
|     3|28.789110867178955|
|     4|  16.6492392099493|
|     2|22.823483351627882|
+------+------------------+



Creating a Managed Table 

In [13]:
table_name = "managed_table_name"

In [15]:
# Save DataFrame as a managed table with overwrite mode
df_bikeshare.write.mode("overwrite").saveAsTable(table_name)

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used
24/03/29 15:11:08 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


In [16]:
df_managedtable = spark.sql(f"SELECT * FROM {table_name} LIMIT 20")
df_managedtable.show()

+-------------------+------+-------+----------+-------+-----+------+--------+---------+------+----------+-----+
|           datetime|season|holiday|workingday|weather| temp| atemp|humidity|windspeed|casual|registered|count|
+-------------------+------+-------+----------+-------+-----+------+--------+---------+------+----------+-----+
|2011-01-01 00:00:00|     1|      0|         0|      1| 9.84|14.395|      81|      0.0|     3|        13|   16|
|2011-01-01 01:00:00|     1|      0|         0|      1| 9.02|13.635|      80|      0.0|     8|        32|   40|
|2011-01-01 02:00:00|     1|      0|         0|      1| 9.02|13.635|      80|      0.0|     5|        27|   32|
|2011-01-01 03:00:00|     1|      0|         0|      1| 9.84|14.395|      75|      0.0|     3|        10|   13|
|2011-01-01 04:00:00|     1|      0|         0|      1| 9.84|14.395|      75|      0.0|     0|         1|    1|
|2011-01-01 05:00:00|     1|      0|         0|      2| 9.84| 12.88|      75|   6.0032|     0|         1

Creating an EXTERNAL table

In [17]:
table_name = "external_table_name"
bucket_name = "pysparkbucket-jm"
table_location = f"gs://{bucket_name}/test"

In [19]:
df_bikeshare.write.option("path", table_location).saveAsTable(table_name, format="parquet", mode="overwrite")


List all the tables that you created

In [20]:
tables = spark.catalog.listTables()

In [21]:
print("Tables in Spark session:")
for table in tables:
    print(table.name)

Tables in Spark session:
external_table_name
managed_table_name
bikeshare_data


Alternative method

In [22]:
spark.sql("SHOW TABLES").show()

+---------+-------------------+-----------+
|namespace|          tableName|isTemporary|
+---------+-------------------+-----------+
|  default|external_table_name|      false|
|  default| managed_table_name|      false|
|         |     bikeshare_data|       true|
+---------+-------------------+-----------+



Dropping a table 

In [23]:
spark.sql("DROP TABLE managed_table_name")

DataFrame[]

In [24]:
spark.sql("SHOW TABLES").show()

+---------+-------------------+-----------+
|namespace|          tableName|isTemporary|
+---------+-------------------+-----------+
|  default|external_table_name|      false|
|         |     bikeshare_data|       true|
+---------+-------------------+-----------+

